In [6]:
import numpy as np
import torch
import torch_geometric
from torch_geometric.nn import VGAE, GCNConv
from torch_geometric.utils import from_networkx, erdos_renyi_graph
import networkx as nx
from scipy.stats import spearmanr

In [10]:
VGAE.__dict__

mappingproxy({'__module__': 'torch_geometric.nn.models.autoencoder',
              '__doc__': 'The Variational Graph Auto-Encoder model from the\n    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_\n    paper.\n\n    Args:\n        encoder (torch.nn.Module): The encoder module to compute :math:`\\mu`\n            and :math:`\\log\\sigma^2`.\n        decoder (torch.nn.Module, optional): The decoder module. If set to\n            :obj:`None`, will default to the\n            :class:`torch_geometric.nn.models.InnerProductDecoder`.\n            (default: :obj:`None`)\n    ',
              '__init__': <function torch_geometric.nn.models.autoencoder.VGAE.__init__(self, encoder: torch.nn.modules.module.Module, decoder: Optional[torch.nn.modules.module.Module] = None)>,
              'reparametrize': <function torch_geometric.nn.models.autoencoder.VGAE.reparametrize(self, mu: torch.Tensor, logstd: torch.Tensor) -> torch.Tensor>,
              'encode': <function torch_g

In [11]:
class Encoder(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, out_channels)
        self.conv_logstd = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

def generate_correlated_params(n, correlation):
    mean = [0, 0]
    cov = [[1, correlation], [correlation, 1]]
    return np.random.multivariate_normal(mean, cov, n)

def generate_graph(n, p):
    return nx.erdos_renyi_graph(n, p)

def get_embedding(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
    return z.mean(dim=0).numpy()


In [14]:
def experiment(num_graphs=100, nodes=50, hidden_channels=32, out_channels=16, correlation=0.5):
    # Generate correlated parameters
    params = generate_correlated_params(num_graphs, correlation)
    params = torch.softmax(torch.tensor(params), dim=1).numpy()  # Normalize using softmax
    # Generate graphs and get VGAE embeddings
    embeddings1 = []
    embeddings2 = []

    for p1, p2 in params:
        # Generate two graphs
        g1 = generate_graph(nodes, p1)
        g2 = generate_graph(nodes, p2)

        # Convert to PyTorch Geometric data objects
        data1 = from_networkx(g1)
        data2 = from_networkx(g2)

        # Add node features (here we use degree as a simple feature)
        data1.x = torch.tensor(list(dict(g1.degree()).values()), dtype=torch.float).view(-1, 1)
        data2.x = torch.tensor(list(dict(g2.degree()).values()), dtype=torch.float).view(-1, 1)

        # Create and train VGAE models
        model1 = VGAE(Encoder(1, hidden_channels, out_channels))
        model2 = VGAE(Encoder(1, hidden_channels, out_channels))

        optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)
        optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.01)

        for epoch in range(100):
            model1.train()
            optimizer1.zero_grad()
            z1 = model1.encode(data1.x, data1.edge_index)
            loss1 = model1.recon_loss(z1, data1.edge_index)
            loss1 += (1 / data1.num_nodes) * model1.kl_loss()
            loss1.backward()
            optimizer1.step()

            model2.train()
            optimizer2.zero_grad()
            z2 = model2.encode(data2.x, data2.edge_index)
            loss2 = model2.recon_loss(z2, data2.edge_index)
            loss2 += (1 / data2.num_nodes) * model2.kl_loss()
            loss2.backward()
            optimizer2.step()

        # Get embeddings
        emb1 = get_embedding(model1, data1)
        emb2 = get_embedding(model2, data2)

        embeddings1.append(emb1)
        embeddings2.append(emb2)

    # Calculate correlations
    param_corr, _ = spearmanr(params[:, 0], params[:, 1])
    emb_corr, _ = spearmanr(np.array(embeddings1).flatten(), np.array(embeddings2).flatten())

    return param_corr, emb_corr

param_corr, emb_corr = experiment()
print(f"Parameter correlation: {param_corr}")
print(f"Embedding correlation: {emb_corr}")